# Model Testing

This file will load the trained model and use it to generate a list of 3 recommended brands for a given user.

In [1]:
import pickle
import pandas as pd
from surprise import KNNBasic

In [2]:
# Load the trained model from the file
with open('model/svd_brand_recommendation_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Define the data file
interaction_data_file = 'data/interaction_data_prepared.csv'

# Import data file into dataframe
interaction_dataframe = pd.read_csv(interaction_data_file)

In [3]:
# Making predictions using the loaded model
def get_top_n_recommendations(user_id, n=3):
    
    # Get all brand ids
    all_brands = set(interaction_dataframe['BRAND_ID'].unique())
    
    # Get brands the user has already interacted with
    interacted_brands = set(interaction_dataframe[interaction_dataframe['USER_ID'] == user_id]['BRAND_ID'].unique())
    
    # Get the list of brands the user has not interacted with
    non_interacted_brands = all_brands - interacted_brands
    
    # Predict ratings for all non-interacted brands
    predictions = []
    for brand_id in non_interacted_brands:
        prediction = loaded_model.predict(user_id, brand_id)
        predictions.append((brand_id, prediction.est))
    
    # Sort predictions by estimated rating in descending order
    predictions.sort(key=lambda x: x[1], reverse=True)
    
    # Get the top N brand recommendations
    top_n_recommendations = predictions[:n]
    
    return top_n_recommendations

In [4]:
# Example usage
user_id = 1480
top_3_recommendations = get_top_n_recommendations(user_id, n=3)

# Print the top 3 brand recommendations for the user
print(f"Top 3 brand recommendations for user {user_id}:")
for brand_id, predicted_rating in top_3_recommendations:
    print(f"Brand ID: {brand_id}, Predicted Rating: {predicted_rating}")

Top 3 brand recommendations for user 1480:
Brand ID: 22615, Predicted Rating: 0.6499789276110525
Brand ID: 27013, Predicted Rating: 0.6302186288103655
Brand ID: 29270, Predicted Rating: 0.6137983658582556
